In [ ]:
from sklearn.linear_model import SGDClassifier

import pandas as pd
import numpy as np

In [ ]:
def make_data(size: float=1e5) -> pd.DataFrame():
    size=int(size)
    
    df = pd.DataFrame(dict(
        a=np.random.randint(10, size=size),
        b=np.random.choice(list('abcd'), size=size)
    ), columns=list('abc'))
    
    def f1(a) -> int:
        return a**2 - a + np.random.randint(4)

    def f2(b) -> int:
        return {k: lambda: np.random.randint(6) for k in list('abcd')}[b]()

    t = df.a.apply(f1) + df.b.apply(f2)
    mi, ma = t.min(), t.max()
    t = 3*(t-mi)/ma
    t = t.astype(int)
    df.c = t.apply(lambda i: 'wxyz'[i])
    
    for label in list('bc'):
        df[label] = df[label].astype('category')
        
    return df

In [ ]:
def get_chunks(df: pd.DataFrame, features=('a', 'b'), target=('c'), buffer: float=1e2):
    i, n = 0, len(df)
    while i < n:
        j = n-1 if i+buffer-1 > n else i+buffer-1
        yield df.loc[i:j, features], df.loc[i:j, target]
        i = j+1

In [ ]:
def process(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for option in df.b.cat.categories:
        out.loc[:, 'b'+'_'+option] = out.loc[:, 'b'].apply(lambda x: x == option).astype(np.uint8)
    del out['b']
    return out

In [ ]:
data = make_data()

In [ ]:
model = SGDClassifier(n_jobs=1)

In [ ]:
model.classes_ = data.c.cat.categories

In [ ]:
for X, y in get_chunks(data):
    X = process(X)
    model.partial_fit(X, y)